### Oracle Vector DB wrapped as a llama-index custom Vector Store

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* Demo on **Medicine Book**
* Demo on Python book

In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID, ENDPOINT

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649


In [3]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [5]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [6]:
v_store = OracleVectorStore(verbose=False)

In [7]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [8]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [9]:
query_engine = index.as_query_engine(similarity_top_k=5)

#### Using the wrapper for the DB Vector Store

In [10]:
question = "Which level of glucose in the bllod are early indication of diabetes?"

In [11]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

#### Use our Vector Store DB

In [12]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 23 ms, sys: 5.35 ms, total: 28.4 ms
Wall time: 697 ms


In [13]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Dod. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

Dod. id: fc1775ef17b774a25a76ab562c8b33f2ba2b59826c13ce78f2b7dbc7872f42f7
Similarity: 0.617
200 Current Essentials of Medicine 7Type 1 Diabetes Mellitus ■Essentials of Diagnosis •Crisp onset, no family history •Polyuria, polydipsia, weight loss •Fasting plasma glucose ≥126 mg/dL; random plasma glucose ≥200 mg/dL with symptoms; glycosuria •Associated with ketosis in untreated state; may present as med-ical emergency (diabetic ketoacidosis) •Long-term risks include retinopathy, nephropathy, neuropathy,and cardiovascular disease ■Differential Diagnosis •Nondiabetic glycosuria (eg, Fanconi’s syndrome) •Diabetes insipidus •Acromegaly •Cushing’s disease or syndrome •Pheochromocytoma •Medications (eg, glucocorticoids, niacin) ■Treatment •Insulin treatment is required •Patient education is crucial, emphasizing dietary management,intensive insulin therapy, self-monitoring of blood glucose, hypo-glycemia awareness, foot and eye care ■Pearl The autoimmune nature of this condition was signaled by 

#### Integrate in the bigger RAG picture

In [14]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print(response.response)
print("")

Question: Which level of glucose in the bllod are early indication of diabetes?
Fasting plasma glucose ≥ 126 mg/dL; random plasma glucose ≥200 mg/dL with symptoms; glycosuria; elevated glycosylated hemoglobin (A1c); ketosis rare

CPU times: user 94.7 ms, sys: 9.49 ms, total: 104 ms
Wall time: 5.07 s
